In [2]:
import pandas as pd

# Load trades
df = pd.read_csv(
    "hood.csv",
    quotechar='"',
    on_bad_lines='skip',  # skips bad lines (use only if okay dropping them)
    engine='python'       # more flexible parser
)

# View columns
print(df.columns)
print(df.head())

Index(['Activity Date', 'Process Date', 'Settle Date', 'Instrument',
       'Description', 'Trans Code', 'Quantity', 'Price', 'Amount'],
      dtype='object')
  Activity Date Process Date Settle Date Instrument  \
0     10/4/2024    10/4/2024   10/7/2024       COIN   
1     10/4/2024    10/4/2024   10/7/2024       HOOD   
2     10/4/2024    10/4/2024   10/7/2024         BA   
3     10/4/2024    10/4/2024   10/7/2024         BA   
4     10/4/2024    10/4/2024   10/7/2024         BA   

                           Description Trans Code Quantity    Price  \
0          COIN 10/11/2024 Put $157.50        STO        1    $2.77   
1  Robinhood Markets\nCUSIP: 770700102       Sell      200   $22.97   
2           BA 10/11/2024 Call $157.50        STO        2    $1.42   
3             Boeing\nCUSIP: 097023105       Sell      100  $152.21   
4           BA 10/11/2024 Call $155.00        STO        1    $2.37   

       Amount  
0     $276.95  
1   $4,593.86  
2     $283.92  
3  $15,220.55  
4  

In [10]:


df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce")
df["Price"] = df["Price"].replace(r'[\$,]', '', regex=True).astype(float)
df["Amount"] = (
    df["Amount"]
    .astype(str)
    .str.replace(r'[\$,]', '', regex=True)
    .str.replace(r'\((.*?)\)', r'-\1', regex=True)
    .astype(float)
)
df = df.dropna(subset=["Quantity", "Price", "Amount"])
# Initialize profit/loss tracking
instrument_stats = {}

for _, row in df.iterrows():
    key = row["Instrument"]
    qty = row["Quantity"]
    amt = row["Amount"]
    price = row["Price"]
    typ = row["Trans Code"]

    if key not in instrument_stats:
        instrument_stats[key] = {
            "total_bought_qty": 0,
            "total_cost": 0.0,
            "realized_pnl": 0.0,
        }

    stats = instrument_stats[key]

    if typ == "Buy":
        stats["total_bought_qty"] += qty
        stats["total_cost"] += amt  # Total cost includes price * qty
    elif typ == "Sell":
        if stats["total_bought_qty"] == 0:
            avg_cost = 0
        else:
            avg_cost = stats["total_cost"] / stats["total_bought_qty"]

        realized = (price - avg_cost) * qty
        stats["realized_pnl"] += realized

        # Adjust cost basis
        stats["total_bought_qty"] -= qty
        stats["total_cost"] -= avg_cost * qty

# Create DataFrame from results
# Round and format large numbers nicely
results["realized_pnl"] = results["realized_pnl"].round(2)
results["total_cost"] = results["total_cost"].round(2)
results["total_bought_qty"] = results["total_bought_qty"].round(2)

# Rename columns for clarity
results.columns = ["Ticker", "Remaining Qty", "Remaining Cost Basis", "Realized Profit/Loss"]

# Optional: format as currency
results["Remaining Cost Basis"] = results["Remaining Cost Basis"].map("${:,.2f}".format)
results["Realized Profit/Loss"] = results["Realized Profit/Loss"].map("${:,.2f}".format)

# Show sorted by P&L
print(results.sort_values("Realized Profit/Loss"))

   Ticker  Remaining Qty                               Remaining Cost Basis  \
20      O       -1009.14  $-1,285,281,843,128,423,659,173,299,398,631,22...   
56   NSYS           0.00                                     $-1,894,255.00   
35   JEPQ       -1008.61                                   $-129,409,838.46   
1    HOOD           0.00                                       $-438,417.12   
52   NVCR           0.00                                        $-48,573.88   
..    ...            ...                                                ...   
53    WIX           0.00                                         $-8,353.75   
54   FTAI           0.00                                           $-799.60   
10    AMD           0.00                                         $-9,228.87   
62   IMOS           0.00                                           $-946.50   
32     SQ        -600.00                                       $-120,008.04   

                                 Realized Profit/Lo